В этом задании мы реализуем другую модель машинного обучения - линейный классификатор. Линейный классификатор подбирает для каждого класса веса, на которые нужно умножить значение каждого признака и потом сложить вместе. Тот класс, у которого эта сумма больше, и является предсказанием модели.
Задание 1.2 - Линейный классификатор (Linear classifier)

В этом задании вы:

потренируетесь считать градиенты различных многомерных функций
реализуете подсчет градиентов через линейную модель и функцию потерь softmax
реализуете процесс тренировки линейного классификатора
подберете параметры тренировки на практике
На всякий случай, еще раз ссылка на туториал по numpy:
http://cs231n.github.io/python-numpy-tutorial/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

from dataset import load_svhn, random_split_train_val
from gradient_check import check_gradient
from metrics import multiclass_accuracy 
import linear_classifer

In [2]:
def prepare_for_linear_classifier(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    # Add another channel with ones as a bias term
    train_flat_with_ones = np.hstack([train_flat, np.ones((train_X.shape[0], 1))])
    test_flat_with_ones = np.hstack([test_flat, np.ones((test_X.shape[0], 1))])    
    return train_flat_with_ones, test_flat_with_ones
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_linear_classifier(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

/tmp/ipykernel_4728/3741198026.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
/tmp/ipykernel_4728/3741198026.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0


In [3]:
# TODO: Implement check_gradient function in gradient_check.py
# All the functions below should pass the gradient check

def square(x):
    return float(x*x), 2*x

check_gradient(square, np.array([3.0]))

def array_sum(x):
    assert x.shape == (2,), x.shape
    return np.sum(x), np.ones_like(x)

check_gradient(array_sum, np.array([3.0, 2.0]))

def array_2d_sum(x):
    assert x.shape == (2,2)
    return np.sum(x), np.ones_like(x)

check_gradient(array_2d_sum, np.array([[3.0, 2.0], [1.0, 0.0]]))

analytic_grad  [6.]
analytic_grad_at_ix  6.0
it value is  3.0
ix  (0,)
zero arr is  [1.e-05]
func1 is  8.999940000099999
func2 is  9.0000600001
numeric_grad_at_ix  6.000000000039306
Gradient check passed!
analytic_grad  [1. 1.]
analytic_grad_at_ix  1.0
it value is  3.0
ix  (0,)
zero arr is  [1.e-05 0.e+00]
func1 is  4.99999
func2 is  5.00001
numeric_grad_at_ix  0.9999999999621422
analytic_grad_at_ix  1.0
it value is  2.0
ix  (1,)
zero arr is  [0.e+00 1.e-05]
func1 is  4.99999
func2 is  5.00001
numeric_grad_at_ix  0.9999999999621422
Gradient check passed!
analytic_grad  [[1. 1.]
 [1. 1.]]
analytic_grad_at_ix  1.0
it value is  3.0
ix  (0, 0)
zero arr is  [[1.e-05 0.e+00]
 [0.e+00 0.e+00]]
func1 is  5.99999
func2 is  6.00001
numeric_grad_at_ix  0.9999999999621422
analytic_grad_at_ix  1.0
it value is  2.0
ix  (0, 1)
zero arr is  [[0.e+00 1.e-05]
 [0.e+00 0.e+00]]
func1 is  5.99999
func2 is  6.00001
numeric_grad_at_ix  0.9999999999621422
analytic_grad_at_ix  1.0
it value is  1.0
ix  (1, 0)


True

In [4]:
a = np.array([[3,2,4,5], [3,3,2,1]])
b = np.array([3,2,90,5])
for elem in b:
    print(elem)

check = np.isclose(a, b, atol=0.5)
print(check)


it = np.nditer(a, flags=['multi_index'], op_flags=['readwrite'])
ix = it.multi_index
it.iternext()
it.iternext()
it.iternext()
it.iternext()
# it.()
print(it.multi_index)

a -= np.max(a)
print("new a is ", a)

3
2
90
5
[[ True  True False  True]
 [ True False False False]]
(1, 0)
new a is  [[-2 -3 -1  0]
 [-2 -2 -3 -4]]


In [5]:
# TODO Implement softmax and cross-entropy for single sample
probs = linear_classifer.softmax(np.array([-10, 0, 10]))

# Make sure it works for big numbers too!
probs = linear_classifer.softmax(np.array([1000, 0, 0]))
assert np.isclose(probs[0], 1.0)

orig predictions  [-10   0  10]
probabilites are  [2.06106005e-09 4.53978686e-05 9.99954600e-01]
orig predictions  [1000    0    0]
probabilites are  [1. 0. 0.]


In [6]:
probs = linear_classifer.softmax(np.array([-5, 0, 5]))
linear_classifer.cross_entropy_loss(probs, 1)

orig predictions  [-5  0  5]
probabilites are  [4.50940412e-05 6.69254912e-03 9.93262357e-01]
loss is  5.006760443547122


5.006760443547122

In [28]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
loss, grad = linear_classifer.softmax_with_cross_entropy(np.array([1, 0, 0]), 1)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, 1), np.array([1, 0, 0], float))

orig predictions  [1 0 0]
probabilites are  [0.57611688 0.21194156 0.21194156]
loss is  1.551444713932051
grad is  [ 0.57611688 -0.78805844  0.21194156]
orig predictions  [1. 0. 0.]
probabilites are  [0.57611688 0.21194156 0.21194156]
loss is  1.551444713932051
grad is  [ 0.57611688 -0.78805844  0.21194156]
analytic_grad  [ 0.57611688 -0.78805844  0.21194156]
analytic_grad_at_ix  0.5761168847658291
it value is  1.0
ix  (0,)
zero arr is  [1.e-05 0.e+00 0.e+00]
orig predictions  [1.00001 0.      0.     ]
probabilites are  [0.57611933 0.21194034 0.21194034]
loss is  1.551450475113109
grad is  [ 0.57611933 -0.78805966  0.21194034]
orig predictions  [0.99999 0.      0.     ]
probabilites are  [0.57611444 0.21194278 0.21194278]
loss is  1.5514389527754138
grad is  [ 0.57611444 -0.78805722  0.21194278]
func1 is  1.5514389527754138
func2 is  1.551450475113109
numeric_grad_at_ix  0.5761168847651099
analytic_grad_at_ix  -0.7880584423829146
it value is  0.0
ix  (1,)
zero arr is  [0.e+00 1.e-05 0.

True

In [29]:
# TODO Extend combined function so it can receive a 2d array with batch of samples
np.random.seed(42)
# Test batch_size = 1
num_classes = 4
batch_size = 1
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(int)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Test batch_size = 3
num_classes = 4
batch_size = 3
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(int)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Make sure maximum subtraction for numberic stability is done separately for every sample in the batch
probs = linear_classifer.softmax(np.array([[20,0,0], [1000, 0, 0]]))
assert np.all(np.isclose(probs[:, 0], 1.0))

orig predictions  [[ 1.  2. -1.  1.]]
max pred is  [2.]
probabilites are  [[0.20603191 0.56005279 0.02788339 0.20603191]]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [20]:
# a = np.array([[[2,3,2,1], [3,42,4,1], [3,4,2,1]], [[2,3,2,1], [3,0,4,1], [3,4,2,1]], [[2,30,2,1], [3,1,4,1], [3,4,2,1]]])
# print(a)
# print("max is \n", np.max(a, axis=2))
# print(a.ndim)


b = np.array([[2,3,2,1], [3,42,4,1], [3,4,2,1]])
c = np.max(b, axis=1)
print("b is \n",b)
print ("c is \n", c)
for elem, i in zip(b, range(b.size)):
    elem -= c[i]
# print("f is ",f )
# b -= np.max(b, axis=1)
print("b is \n",b)
# probabilities = np.zeros_like(predictions)
# sum_exps = np.sum(np.exp(b))
# probabilities = np.exp(b)/sum_exps
# print("probs are ", probabilities)

[[[ 2  3  2  1]
  [ 3 42  4  1]
  [ 3  4  2  1]]

 [[ 2  3  2  1]
  [ 3  0  4  1]
  [ 3  4  2  1]]

 [[ 2 30  2  1]
  [ 3  1  4  1]
  [ 3  4  2  1]]]
max is 
 [[ 3 42  4]
 [ 3  4  4]
 [30  4  4]]
3
b is 
 [[ 2  3  2  1]
 [ 3 42  4  1]
 [ 3  4  2  1]]
c is 
 [ 3 42  4]
b is 
 [[ -1   0  -1  -2]
 [-39   0 -38 -41]
 [ -1   0  -2  -3]]
